In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import gc
import json

In [2]:
dtypes = {
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'hour'          : 'uint8',
        'is_attributed' : 'uint8', 
        'ip_day_hour_count': 'uint32', 
        'ip_os_day_hour_count': 'uint32', 
        'ip_app_day_hour_count': 'uint32', 
        'ip_app_os_day_hour_count': 'uint32', 
        'app_day_hour_count': 'uint32', 
        'ip_device_os_count': 'uint32', 
        'ip_app_device_os_count': 'uint32', 
        'ip_device_os_mean': 'float16',
        'ip_app_device_os_mean': 'float16',
        'ip_app_device_mean': 'float16',
        'app_device_os_mean': 'float16',
        'ip_device_os_time2nextclick': 'int32',
        'ip_app_device_os_time2nextclick': 'int32',
        'ip_app_device_time2nextclick': 'int32',
        'ip_device_os_time2previousclick': 'int32',
        'ip_app_device_os_time2previousclick': 'int32',
        'ip_app_device_time2previousclick': 'int32',
        'ip_device_os_countfromfuture': 'uint32', 
        'ip_app_device_os_countfromfuture': 'uint32', 
        'ip_app_device_countfromfuture': 'uint32', 
        'ip_device_os_countfrompast': 'uint32', 
        'ip_app_device_os_countfrompast': 'uint32', 
        'ip_app_device_countfrompast': 'uint32', 
        'ip_device_os_lasttimediff': 'int32',
        'ip_app_device_os_lasttimediff': 'int32',
        'ip_app_device_lasttimediff': 'int32',
        'ip_device_os_firsttimediff': 'int32',
        'ip_app_device_os_firsttimediff': 'int32',
        'ip_app_device_firsttimediff': 'int32',
        'matrixFact_user_iposdeviceapp_item_app': 'float16',
        'matrixFact_user_ip_item_appdeviceos': 'float16',
        'matrixFact_user_ipchannel_item_appdeviceos': 'float16',
        'ip_device_os_regression': 'float16',
        'ip_app_device_os_regression': 'float16',
        'ip_app_device_regression': 'float16',
        'ip_app_device_os_channel_regression': 'float16', 
        'attributed_timediffmax':'int32',
        'attributed_timediffmin':'int32',
        'attributed_timediff':'float16',
        'matrixFact_user_ipappdeviceos_item_channel': 'float16'
    
        } 

In [3]:
# load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
# file_format = '{}_features_supplementV3_feature42.csv'
# day_list = ['day7', 'day8', 'day9']
# df_dict = {}
# df_extra = {}
# for file in ['day7', 'day8', 'day9']: 
#     print(file)
#     df_dict[file] = pd.read_csv(load_path+file_format.format(file),dtype=dtypes)
    
# target = 'is_attributed'
# feature_cols = list(set(df_dict['day7'].columns) - set([target]))


In [4]:
df_all = pd.read_csv('/home/kai/data/kaggle/talkingdata/wl/data/equalhour/ALL_features_supplementV3_feature42.csv',dtype=dtypes)
target = 'is_attributed'
feature_cols = list(set(df_all.columns) - set([target]))

# define parameter

In [5]:
target = 'is_attributed'

day_list = ['day7', 'day8', 'day9']

combine = 0
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 2400,
        'learning_rate': 0.05,
        'num_leaves': 61,
        'num_threads': 4, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 390,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.5, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':60,
        'bagging_fraction': 0.7, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        'verbose': 0,
        'scale_pos_weight': 400,
        'metric' : [ 'auc']
    }

categorical_col = [ 'app', 'device', 'os', 'channel', 'hour']

In [6]:

extra = set(['ip_device_os_countself',
             'ip_day_hour_countself', 
             'app_day_hour_countself',
             'ip_app_device_os_countself',
             'ip_app_day_hour_countself', 
             'ip_os_day_hour_countself', 
             'ip_app_os_day_hour_countself',
             'ip_device_os_regression',
             'ip_app_device_os_regression', 
             'ip_app_device_regression',
             'ip_app_device_os_channel_regression' ])


feature_cols = list(set(feature_cols) - extra)

In [7]:
feature_cols

['ip_app_os_day_hour_count',
 'ip_app_device_os_time2nextclick',
 'ip_app_device_countfrompast',
 'os',
 'app_device_os_mean',
 'ip_app_device_os_time2previousclick',
 'ip_device_os_countfrompast',
 'device',
 'ip_device_os_time2previousclick',
 'ip_os_day_hour_count',
 'ip_app_device_os_lasttimediff',
 'ip_app_device_time2previousclick',
 'ip_app_device_os_countfromfuture',
 'matrixFact_user_iposdeviceapp_item_app',
 'ip_app_device_lasttimediff',
 'ip_app_device_os_count',
 'ip_app_device_countfromfuture',
 'attributed_timediff',
 'app',
 'ip_app_device_firsttimediff',
 'app_day_hour_count',
 'matrixFact_user_ipchannel_item_appdeviceos',
 'hour',
 'ip_app_device_time2nextclick',
 'ip_device_os_lasttimediff',
 'ip_app_device_os_countfrompast',
 'ip_app_day_hour_count',
 'ip_device_os_mean',
 'channel',
 'ip_app_device_os_firsttimediff',
 'ip_device_os_count',
 'ip_app_device_mean',
 'ip_day_hour_count',
 'matrixFact_user_ip_item_appdeviceos',
 'ip_device_os_time2nextclick',
 'ip_device

In [8]:
def train_lightgbm(x_train, x_val, feature_cols, categorical_feature, params, best_round = None, target='is_attributed'):
    param = params.copy()
    y_train = x_train[target].values
    y_val = x_val[target].values
    
    lgb_train = lgb.Dataset(x_train[feature_cols], y_train, categorical_feature = categorical_feature)
    lgb_val = lgb.Dataset(x_val[feature_cols], y_val, categorical_feature = categorical_feature)
    if best_round is not None:
        param['num_rounds'] = best_round
        del param['early_stopping_round']
    print('start training')
    model = lgb.train(param, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)
    return model

In [9]:
best_round_list = []
model_path = '/home/kai/data/kaggle/talkingdata/wl/data/lightgbm/'
prefix = 'paramsCombo_{}_lr0.05_scale400_ntree500_ff0.5_bf0.7'

    
best_round_arr = np.array(best_round_list)
model = train_lightgbm(df_all, df_all.iloc[-1000000:], feature_cols, categorical_col, params, best_round=500)
model.save_model(model_path+prefix.format(1))
# np.save(model_path+prefix.format(1)+'.npy', best_round_arr)

import json
feature_file = '/home/kai/data/kaggle/talkingdata/wl/data/lightgbm/featurecolsV3_col38_scale99.json'

with open(feature_file, 'w') as outfile:
    json.dump(feature_cols, outfile)
    
# model1 = lgb.Booster(model_file='gbm_test',)
    
    
    

start training


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


[10]	valid_0's auc: 0.986985
[20]	valid_0's auc: 0.98878
[30]	valid_0's auc: 0.9896
[40]	valid_0's auc: 0.990608
[50]	valid_0's auc: 0.991212
[60]	valid_0's auc: 0.991753
[70]	valid_0's auc: 0.992197
[80]	valid_0's auc: 0.992481
[90]	valid_0's auc: 0.992765
[100]	valid_0's auc: 0.993124
[110]	valid_0's auc: 0.993489
[120]	valid_0's auc: 0.993717
[130]	valid_0's auc: 0.994229
[140]	valid_0's auc: 0.994467
[150]	valid_0's auc: 0.994675
[160]	valid_0's auc: 0.994782
[170]	valid_0's auc: 0.994844
[180]	valid_0's auc: 0.994982
[190]	valid_0's auc: 0.9951
[200]	valid_0's auc: 0.995624
[210]	valid_0's auc: 0.995716
[220]	valid_0's auc: 0.995766
[230]	valid_0's auc: 0.995866
[240]	valid_0's auc: 0.996062
[250]	valid_0's auc: 0.996166
[260]	valid_0's auc: 0.996202
[270]	valid_0's auc: 0.996322
[280]	valid_0's auc: 0.996521
[290]	valid_0's auc: 0.996559
[300]	valid_0's auc: 0.996551
[310]	valid_0's auc: 0.996599
[320]	valid_0's auc: 0.996684
[330]	valid_0's auc: 0.996708
[340]	valid_0's auc: 0.9

In [10]:
# best_round_list = []
# model_path = '/home/kai/data/kaggle/talkingdata/wl/data/lightgbm/'
# prefix = 'paramsCombo_{}_lr0.03'
# for day in day_list:
#     train_day = list(set(day_list)-set([day]))
#     trainset = pd.concat([df_dict[train_day[0]],df_dict[train_day[1]]])
#     valset = df_dict[day]
#     print('building train val done!')
#     model = train_lightgbm(trainset, valset, feature_cols, categorical_col, params)
#     best_round = model.best_iteration
#     best_round_list.append(best_round)
    
# df_all = pd.concat([df_dict['day7'], df_dict['day8'], df_dict['day9']])
# del df_dict
# gc.collect()
    
# best_round_arr = np.array(best_round_list)
# model = train_lightgbm(df_all, valset, feature_cols, categorical_col, params, best_round=1200)
# model.save_model(model_path+prefix.format(1))
# np.save(model_path+prefix.format(1)+'.npy', best_round_arr)

# import json
# feature_file = '/home/kai/data/kaggle/talkingdata/wl/data/lightgbm/featurecolsV3_col38.json'

# with open(feature_file, 'w') as outfile:
#     json.dump(feature_cols, outfile)
    
# # model1 = lgb.Booster(model_file='gbm_test',)
    
    
    

# Load Model and predict

In [11]:
model_path = '/home/kai/data/kaggle/talkingdata/wl/data/lightgbm/gridsearch/'
feature_file = '/home/kai/data/kaggle/talkingdata/wl/data/lightgbm/gridsearch/lightgbm-featurecolsV3_col38_lightgbm_bestparam_bestround_225_roc_0.9824899830726912.json'
prefix = 'lightgbm_bestparam_bestround_225_roc_0.9824899830726912'
modelfile=model_path+prefix
model_load = lgb.Booster(model_file=modelfile)
print('load model done!')

# this may not need to be loaded in the future
# num_trees_load = np.load(model_path+prefix.format(1)+'.npy')
num_trees_load = [630, 680, 720]
print('load number of trees done!')

load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
file_format = '{}_features_supplementV3_feature42.csv'
test = pd.read_csv(load_path+file_format.format('test'),dtype=dtypes)
print('load test feature done!')

feature_col_load = json.load(open(feature_file))
print('load feature cols done!')

preds = []
submission_postfix = '_ntree'

for ntree in num_trees_load:
    ntree = int(ntree)
    preds.append(model_load.predict(test[model_load.feature_name()], num_iteration=ntree))
    print('predicting... {} done!'.format(ntree))
    submission_postfix += '_{}'.format(ntree)
pred = np.mean(preds, axis=0)


print('getting submission')
df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
print('loading file done!')
df_sub = pd.DataFrame()
df_sub['click_id'] = df_test_raw['click_id']
df_sub['is_attributed'] = pred
print('predicting file done!')

# prefix = 'paramsCombo_{}_lr0.05_scale99_ntree500_ff0.5_bf0.7_onehot500'
submission_name = model_path+prefix+submission_postfix+'.csv.gz'
df_sub.to_csv(submission_name, compression='gzip', index=False)


np.save(model_path+prefix+submission_postfix+'.npy', pred)
    

load model done!
load number of trees done!
load test feature done!
load feature cols done!
predicting... 550 done!
predicting... 620 done!
predicting... 700 done!
getting submission
loading file done!
predicting file done!


In [11]:
# modelfile=model_path+prefix.format(1)
# model_load = lgb.Booster(model_file=modelfile)
# print('load model done!')

# # this may not need to be loaded in the future
# # num_trees_load = np.load(model_path+prefix.format(1)+'.npy')
# num_trees_load = [300, 370, 470]
# print('load number of trees done!')

# load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
# file_format = '{}_features_supplementV3_feature42.csv'
# test = pd.read_csv(load_path+file_format.format('test'),dtype=dtypes)
# print('load test feature done!')


# feature_col_load = json.load(open(feature_file))
# print('load feature cols done!')

# preds = []
# submission_postfix = 'ntree'

# for ntree in num_trees_load:
#     ntree = int(ntree)
#     preds.append(model_load.predict(test[feature_col_load], num_iteration=ntree))
#     print('predicting... {} done!'.format(ntree))
#     submission_postfix += '_{}'.format(ntree)
# pred = np.mean(preds, axis=0)


# print('getting submission')
# df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
# print('loading file done!')
# df_sub = pd.DataFrame()
# df_sub['click_id'] = df_test_raw['click_id']
# df_sub['is_attributed'] = pred
# print('predicting file done!')
# submission_name = model_path+prefix.format(1)+submission_postfix+'.csv.gz'
# df_sub.to_csv(submission_name, compression='gzip', index=False)

    

load model done!
load number of trees done!
load test feature done!
load feature cols done!
predicting... 300 done!
predicting... 370 done!
predicting... 470 done!
getting submission
loading file done!
predicting file done!


# Average

In [2]:
#### load each and average
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
day7 = np.load(load_path+'concat_day8day9_val_day7_allday_supplement_V2_attributediff_matchannel.npy')
day8 = np.load(load_path+'concat_day7day9_val_day8_allday_supplement_V2_attributediff_matchannel.npy')
day9 = np.load(load_path+'concat_day7day8_val_day9_allday_supplement_V2_attributediff_matchannel.npy')

pred = (day7 + day8 + day9)/3

In [3]:
# prediction
df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
print('loading file done!')
df_sub = pd.DataFrame()
df_sub['click_id'] = df_test_raw['click_id']
df_sub['is_attributed'] = pred
print('predicting file done!')
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/allday_blend_supplement_V2_attributediff_matchannel.csv.gz', compression='gzip', index=False)


loading file done!
predicting file done!
